# **PART 5:** Cluster annotation & visualization

In [1]:
import os
# Set the working directory to the repository directory
os.chdir("/home/sebastiaan/PhD/Repositories/book_chapter/")

In [2]:
import pandas as pd

from src.tools import merge_results, _get_sequences

In [10]:
# TCRex results
tcrex = pd.read_csv("./results/tcrex/P1_0.tsv", sep = "\t")
# ClusTCR results
clust = pd.read_csv("./results/clustcr/P1_0_8.txt", sep = "\t")
# Original data
rawdata = pd.read_csv("./data/examples/P1_0_clones.txt", sep = "\t")

merged = merge_results(
    original_data = rawdata,
    clustering_results = clust,
    tcrex_results = tcrex
    )

merged = merged.dropna(subset = ["cluster"])

keep = ["duplicate_count",
        "frequency",
        "junction_aa", 
        "v_call", 
        "j_call", 
        "cluster", 
        "cluster", 
        "epitope"]

merged = merged[keep]
merged.to_csv("./results/P1_0_clusters_tcrex.tsv", sep = "\t", index = False)

In [9]:
# TCRex results
tcrex = pd.read_csv("./results/tcrex/P1_15.tsv", sep = "\t")
# ClusTCR results
clust = pd.read_csv("./results/clustcr/P1_15_8.txt", sep = "\t")
# Original data
rawdata = pd.read_csv("./data/examples/P1_15_clones.txt", sep = "\t")

merged = merge_results(
    original_data = rawdata,
    clustering_results = clust,
    tcrex_results = tcrex
    )

merged = merged.dropna(subset = ["cluster"])

keep = ["duplicate_count",
        "frequency",
        "junction_aa", 
        "v_call", 
        "j_call", 
        "cluster", 
        "cluster", 
        "epitope"]

merged = merged[keep]
merged.to_csv("./results/P1_15_clusters_tcrex.tsv", sep = "\t", index = False)

,duplicate_count,frequency,junction_aa,v_call,j_call,cluster,cluster,epitope
0,1480.0,7.687245e-04,CASSDRSGGADEQFF,TRBV6-4*00,TRBJ2-1*00,265.0,265.0,NaN
1,857.0,4.451331e-04,CASSEDPRLQRDEQFF,TRBV6-1*00,TRBJ2-1*00,121.0,121.0,NaN
2,687.0,3.568336e-04,CASSDSPTGAYGYTF,TRBV6-1*00,TRBJ1-2*00,504.0,504.0,NaN
3,558.0,2.898299e-04,CASSERENTEAFF,TRBV6-4*00,TRBJ1-1*00,613.0,613.0,NaN
4,3.0,1.558225e-06,CASSERENTEAFF,TRBV6-1*00,TRBJ1-1*00,613.0,613.0,NaN
...,...,...,...,...,...,...,...,...
75518,1.0,5.194085e-07,CASSKGSHQRHEQFF,TRBV21-1*00,TRBJ2-1*00,15512.0,15512.0,NaN
75519,1.0,5.194085e-07,CASHTVGSSETQYF,TRBV21-1*00,TRBJ2-5*00,15530.0,15530.0,NaN
75520,1.0,5.194085e-07,CARSKTLTGETQYF,TRBV21-1*00,TRBJ2-5*00,15514.0,15514.0,NaN
75521,1.0,5.194085e-07,CASSKRQTYEQYF,TRBV21-1*00,TRBJ2-7*00,15520.0,15520.0,NaN
